<a href="https://colab.research.google.com/github/akhilnooney/Akhil/blob/master/Covid19_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install tweepy

In [0]:
pip install geotext

     |████████████████████████████████| 2.0MB 2.8MB/s 


In [0]:

import tweepy
import pandas as pd
from geotext import GeoText
# API credentials here
consumer_key = 'hi3NdqgmcADqcZPRtHYxVxgZz'
consumer_secret = '3GkNzsZtQTpxl32jMm9Pg6YwustmvlRdzQXQ4GhwXQvTr1BObu'
access_token = '815976441619693568-XapVuD7GFviOKanvhCZOFXLA5Yezs5G'
access_token_secret = '93RUpa40OzRYZyTVemF9cy6hs8xShCSgv7X9UcUmE5yx3'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
searchString = '#covid_19' or '#covid-19'
cursor = tweepy.Cursor(api.search, q=searchString, count=20, lang="en", tweet_mode='extended')
tweets=[]
maxCount = 3000
count = 0
df = pd.DataFrame({'id': [], 'Created_at': [], 'Tweet': [],'Location':[]})
for tweet in cursor.items():    
    if(tweet.user.location!=''):
      places = GeoText(tweet.user.location)
      df = df.append({'id': tweet.id, 'Created_at': tweet.created_at, 'Tweet':tweet.full_text,'Location':places.countries}, ignore_index=True)
      count = count + 1
      if(count == maxCount):
        break;
df

Rate limit reached. Sleeping for: 853


,id,Created_at,Tweet,Location
0,1.246565e+18,2020-04-04 22:25:25,RT @WarRoomPandemic: Dr. Grace: Most #NYC hosp...,[]
1,1.246565e+18,2020-04-04 22:25:21,RT @MichaelTCucek: #JapanSociety - the country...,[]
2,1.246564e+18,2020-04-04 22:25:10,"Our Executive Director, @eedgerly, wanted to u...",[]
3,1.246564e+18,2020-04-04 22:25:06,RT @medmalinf: [#COVID19] No Evidence of Rapid...,[]
4,1.246564e+18,2020-04-04 22:24:57,Shopaholic? Here's your fix.\nsanipoe 360 Make...,[]
...,...,...,...,...
2995,1.246520e+18,2020-04-04 19:26:41,Here's some of the information on JCAN that is...,[]
2996,1.246520e+18,2020-04-04 19:26:40,RT @ramichuene: I’m not trying to start a figh...,[]
2997,1.246520e+18,2020-04-04 19:26:39,RT @revathitweets: Goshamahal MLA from BJP @Ti...,[India]
2998,1.246520e+18,2020-04-04 19:26:39,RT @HarsimratBadal_: As I relive the mental ag...,[]


In [0]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [

True

In [0]:
#Tokenization
def identify_tokens(row):
    review = row['Tweet']
    tokens = nltk.word_tokenize(review)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

In [0]:
#Stemming
from nltk.stem import PorterStemmer
stemming = PorterStemmer()
def stem_list(row):
    my_list = row['Tweet']
    stemmed_list = [stemming.stem(word) for word in my_list]
    return (stemmed_list)

In [0]:
#Lemmatization
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
def lem_list(row):
    my_list = row['Tweet']
    lemmatized_list = [lemmatizer.lemmatize(word) for word in my_list]
    return (lemmatized_list)

In [0]:
#Removing stop words
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))                  

def remove_stops(row):
    my_list = row['Tweet']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

In [0]:
#Rejoining Tweets
def rejoin_words(row):
    my_list = row['Tweet']
    joined_words = ( " ".join(my_list))
    return joined_words

In [0]:
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
frame=df[(df.astype(str)['Location']!='[]')]
frame['Tweet'] = frame['Tweet'].str.lower()
frame = frame.drop_duplicates(subset='Tweet')
frame["Tweet"] = frame['Tweet'].str.replace('[^\w\s]','')
frame['Tweet'] = frame.apply(identify_tokens, axis=1)
frame['Tweet'] = frame.apply(stem_list, axis=1)
frame['Tweet'] = frame.apply(lem_list, axis=1)
frame['Tweet'] = frame.apply(remove_stops, axis=1)
frame['Tweet'] = frame.apply(rejoin_words, axis=1)
frame['Tweet'] = frame['Tweet'].str.replace("rt",'')
frame['Location']=frame['Location'].str.get(0)
Top_countries=frame['Location'].value_counts()[:5].sort_values(ascending=False).index.tolist()
frame=frame.loc[frame['Location'].isin(Top_countries)]
frame

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,id,Created_at,Tweet,Location
23,1.246564e+18,2020-04-04 22:23:25,african leader confer call discus old french ...,South Africa
27,1.246564e+18,2020-04-04 22:23:13,norbeelek sa man vavi show u video fight coron...,South Africa
49,1.246564e+18,2020-04-04 22:21:32,adriandelmont yet want test vaccin africa peop...,South Africa
59,1.246563e+18,2020-04-04 22:21:00,mohammadkaif good hand ensur catch ball clean...,India
63,1.246563e+18,2020-04-04 22:20:46,thi best thing ive seen dure thi pandem phine...,United Kingdom
...,...,...,...,...
2926,1.246520e+18,2020-04-04 19:29:32,meat eater risk sta danger diseas like corona...,United Kingdom
2943,1.246520e+18,2020-04-04 19:28:55,dolphin pop surfac say hello nearli yr onli ca...,India
2971,1.246520e+18,2020-04-04 19:27:34,drmurukada noconvers suggest dump dablighijam...,India
2985,1.246520e+18,2020-04-04 19:27:02,seed inherit buri conscienc awesom youtub vide...,South Africa
